In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, preprocessing
import pandas as pd
from matplotlib import style
import statistics
from collections import Counter
from IPython.display import display

In [3]:
how_much_better = 8

FEATURES =  ['DE Ratio',
             'Trailing P/E',
             'Price/Sales',
             'Price/Book',
             'Profit Margin',
             'Operating Margin',
             'Return on Assets',
             'Return on Equity',
             'Revenue Per Share',
             'Market Cap',
             'Enterprise Value',
             'Forward P/E',
             'PEG Ratio',
             'Enterprise Value/Revenue',
             'Enterprise Value/EBITDA',
             'Revenue',
             'Gross Profit',
             'EBITDA',
             'Net Income Avl to Common ',
             'Diluted EPS',
             'Earnings Growth',
             'Revenue Growth',
             'Total Cash',
             'Total Cash Per Share',
             'Total Debt',
             'Current Ratio',
             'Book Value Per Share',
             'Cash Flow',
             'Beta',
             'Held by Insiders',
             'Held by Institutions',
             'Shares Short (as of',
             'Short Ratio',
             'Short % of Float',
             'Shares Short (prior ']

In [4]:
def Status_Calc(stock, sp500):
    difference = stock - sp500
    
    if difference > how_much_better:
        return 1
    else:
        return 0

In [5]:
def Build_Data_Set():
    data_df = pd.DataFrame.from_csv("key_stats_acc_perf_NO_NA_enhanced(1).csv")

    #data_df = data_df[:100]
    data_df = data_df.reindex(np.random.permutation(data_df.index))
    data_df = data_df.replace("NaN",0).replace("N/A",0)
    data_df = data_df.fillna(0)
    
    data_df["Status2"] = list(map(Status_Calc, data_df["stock_p_change"], data_df["sp500_p_change"]))    
    

    X = np.array(data_df[FEATURES].values)

    y = (data_df["Status2"]
         .replace("underperform",0)
         .replace("outperform",1)
         .values.tolist())

    X = preprocessing.scale(X)
    Z = np.array(data_df[['stock_p_change', 'sp500_p_change']])


    return X,y,Z

In [6]:
def Analysis():

    test_size = 1
    invest_amount = 10000
    total_invests = 0
    if_market = 0
    if_strat = 0
    
    
    X, y, Z = Build_Data_Set()

    clf = svm.SVC(kernel="linear", C= 1.0)
    clf.fit(X[:-test_size],y[:-test_size])

    correct_count = 0
    prediction = clf.predict(X)

    for x in range(1, test_size+1):
        if prediction[x] == y[x]:
            correct_count += 1
            
        if prediction[x] == 1:
            invest_return = invest_amount + (invest_amount * Z[x][0]/100 )
            market_return = invest_amount + (invest_amount * Z[x][1]/100 )
            total_invests += 1
            if_market += market_return
            if_strat += invest_return
            

#    print("Accuracy:", (float(correct_count)/float(test_size)) * 100.00)
#    print("Total trades:", total_invests)
#    print("Ending with strategy:", if_strat)
#    print("Ending with market:", if_market)
#    compared = ((if_strat - if_market) / if_market) * 100.0
#    do_nothing = total_invests * invest_amount
#    avg_market = ((if_market - do_nothing) / do_nothing) * 100.0
#    avg_strat = ((if_strat - do_nothing) / do_nothing) * 100.0
#    print("compared to market we earn " + str(compared) + "% more")
#    print("Average market return " + str(avg_market) + "%")
#    print("Average stratergy return " + str(avg_strat) + "%")
    
    data_df = pd.DataFrame.from_csv("forward_sample_NO_NA.csv")
    data_df = data_df.replace("NaN",0).replace("N/A",0)
    data_df = data_df.fillna(0)
    X = np.array(data_df[FEATURES].values)

    X = preprocessing.scale(X)

    Z = data_df["Ticker"].values.tolist()

    invest_list = []

    for i in range(len(X)):
        p = clf.predict([X[i]])[0]
        if p == 1:
            #print(Z[i])
            invest_list.append(Z[i])
 
    return invest_list

final_list = []
investing_list = []
loops = 8

for x in range(loops):
    stock_list = Analysis()
    for e in stock_list:
        final_list.append(e)
        
x = Counter(final_list)
print( '--' * 62)

for each in x:
    if x[each] > loops - (loops/3):
        investing_list.append(each)
        
print("Number of investments:", len(investing_list))
print( '--' * 62)
print(investing_list)        
    
dfa = pd.DataFrame(investing_list, columns=["investing_list"])
dfa.to_csv('reccomendation.csv', index=False)
print("investing_list is saved as csv file")

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


----------------------------------------------------------------------------------------------------------------------------
Number of investments: 5
----------------------------------------------------------------------------------------------------------------------------
['c', 'eog', 'fmc', 'kmi', 'wat']
investing_list is saved as csv file
